Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [11]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [12]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [13]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [14]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [17]:
num_steps = 1001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.802186
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.410265
Minibatch accuracy: 56.2%
Validation accuracy: 52.9%
Minibatch loss at step 100: 0.725040
Minibatch accuracy: 75.0%
Validation accuracy: 68.6%
Minibatch loss at step 150: 0.216434
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 200: 0.471942
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 250: 0.594136
Minibatch accuracy: 93.8%
Validation accuracy: 78.9%
Minibatch loss at step 300: 0.655604
Minibatch accuracy: 75.0%
Validation accuracy: 80.6%
Minibatch loss at step 350: 0.675222
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.709126
Minibatch accuracy: 75.0%
Validation accuracy: 81.9%
Minibatch loss at step 450: 0.981384
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.375542
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [19]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [20]:
num_steps = 1001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.164816
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.123319
Minibatch accuracy: 25.0%
Validation accuracy: 16.3%
Minibatch loss at step 100: 0.995736
Minibatch accuracy: 68.8%
Validation accuracy: 53.9%
Minibatch loss at step 150: 0.414293
Minibatch accuracy: 87.5%
Validation accuracy: 69.2%
Minibatch loss at step 200: 0.707517
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 250: 0.515735
Minibatch accuracy: 93.8%
Validation accuracy: 77.7%
Minibatch loss at step 300: 0.708787
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.603421
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Minibatch loss at step 400: 0.707188
Minibatch accuracy: 75.0%
Validation accuracy: 81.2%
Minibatch loss at step 450: 0.960006
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.419424
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The next model is similar with Lenet 

In [31]:
batch_size = 16
patch_size = 5
depth_1 = 6
depth_2 = 16
num_hidden_1 = 128
num_hidden_2 = 64
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden_1], stddev=0.1))

  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_1, num_hidden_2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Mode2.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.avg_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden_4 = tf.nn.relu(tf.matmul(hidden_3, layer4_weights) + layer4_biases)
    return tf.matmul(hidden_4, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [32]:
num_steps = 20001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.106734
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.852274
Minibatch accuracy: 43.8%
Validation accuracy: 24.2%
Minibatch loss at step 100: 1.035018
Minibatch accuracy: 56.2%
Validation accuracy: 55.4%
Minibatch loss at step 150: 0.543365
Minibatch accuracy: 81.2%
Validation accuracy: 66.3%
Minibatch loss at step 200: 0.660281
Minibatch accuracy: 87.5%
Validation accuracy: 74.3%
Minibatch loss at step 250: 0.675928
Minibatch accuracy: 81.2%
Validation accuracy: 75.1%
Minibatch loss at step 300: 0.662538
Minibatch accuracy: 81.2%
Validation accuracy: 77.2%
Minibatch loss at step 350: 0.626301
Minibatch accuracy: 81.2%
Validation accuracy: 77.1%
Minibatch loss at step 400: 0.952647
Minibatch accuracy: 62.5%
Validation accuracy: 77.1%
Minibatch loss at step 450: 1.253603
Minibatch accuracy: 56.2%
Validation accuracy: 77.5%
Minibatch loss at step 500: 0.345724
Minibatch accuracy: 93.8%
Validation accuracy: 75.5%
Mi

Then I add Dropout, and learning rate decay.

In [54]:
batch_size = 16
patch_size = 5
depth_1 = 6
depth_2 = 16
num_hidden_1 = 128
num_hidden_2 = 64
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden_1], stddev=0.1))

  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_1, num_hidden_2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  global_step = tf.Variable(0)
  # Mode3.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.avg_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.avg_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    drop_1 = tf.nn.dropout(hidden_3,0.5)
    hidden_4 = tf.nn.relu(tf.matmul(drop_1, layer4_weights) + layer4_biases)
    drop_2 = tf.nn.dropout(hidden_4,0.5)
    return tf.matmul(drop_2, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #learning_rate = tf.train.exponential_decay(0.01, global_step, 1000, 0.85, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [55]:
num_steps = 20001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.511963
Minibatch accuracy: 12.5%
Validation accuracy: 9.9%
Minibatch loss at step 500: 1.369457
Minibatch accuracy: 43.8%
Validation accuracy: 59.2%
Minibatch loss at step 1000: 0.936199
Minibatch accuracy: 81.2%
Validation accuracy: 69.6%
Minibatch loss at step 1500: 0.762804
Minibatch accuracy: 75.0%
Validation accuracy: 74.8%
Minibatch loss at step 2000: 1.029085
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.681567
Minibatch accuracy: 81.2%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 1.271703
Minibatch accuracy: 50.0%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 0.831516
Minibatch accuracy: 68.8%
Validation accuracy: 78.7%
Minibatch loss at step 4000: 0.839705
Minibatch accuracy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 0.671504
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 5000: 0.549689
Minibatch accuracy: 81.2%
Validation accuracy

I find that adding dropout couldn't improve the accuracy, so I remove the dropout layer and only add learning rate decay.

In [63]:
batch_size = 16
patch_size = 5
depth_1 = 6
depth_2 = 16
num_hidden_1 = 128
num_hidden_2 = 64
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden_1], stddev=0.1))

  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_1]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_1, num_hidden_2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  global_step = tf.Variable(0)
  # Mode3.
  def model(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.avg_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.avg_pool(hidden_2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden_3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden_4 = tf.nn.relu(tf.matmul(hidden_3, layer4_weights) + layer4_biases)
    return tf.matmul(hidden_4, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 2000, 0.97, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [64]:
num_steps = 40001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.079184
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.538776
Minibatch accuracy: 81.2%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 0.663574
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 0.500340
Minibatch accuracy: 87.5%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 0.793911
Minibatch accuracy: 75.0%
Validation accuracy: 83.7%
Minibatch loss at step 2500: 0.527557
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 1.175681
Minibatch accuracy: 62.5%
Validation accuracy: 85.4%
Minibatch loss at step 3500: 0.320156
Minibatch accuracy: 93.8%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.960698
Minibatch accuracy: 75.0%
Validation accuracy: 86.1%
Minibatch loss at step 4500: 0.328867
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 5000: 0.266736
Minibatch accuracy: 93.8%
Validation accurac

The accuracy is nice, and I find that The accuracy is nice, and I find that choosing a nice learning rate is very very important and it can affect accuracy a lot. I choose a lot of learning rate and many of them couldn't do well. Especially, a big learning rate is very bad, so choose a small learning rate.